In [1]:
import tensorflow as tf
import matplotlib.image as img
%matplotlib inline
import numpy as np
from collections import defaultdict
import collections
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models
import cv2

Dataset is stored on shared google drive.

In [11]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Create .txt files with image filenames. Separate file for original and fake images in train and test datasets.

In [13]:
!ls /content/drive/Shareddrives/datasets/animals/data/train/original/*.jpg > /content/drive/Shareddrives/datasets/animals/originals_train.txt
!ls /content/drive/Shareddrives/datasets/animals/data/train/fake/*.jpg > /content/drive/Shareddrives/datasets/animals/fakes_train.txt

!ls /content/drive/Shareddrives/datasets/animals/data/test/original/*.jpg > /content/drive/Shareddrives/datasets/animals/originals_test.txt
!ls /content/drive/Shareddrives/datasets/animals/data/test/fake/*.jpg > /content/drive/Shareddrives/datasets/animals/fakes_test.txt

path = "/content/drive/Shareddrives/datasets/animals/"
original_train_filenames_path = path + "originals_train.txt"
fake_train_filenames_path = path + "fakes_train.txt"
original_test_filenames_path = path + "originals_test.txt"
fake_test_filenames_path = path + "fakes_test.txt"

Paths to folders that include images. Current structure (after copying from previous notebook):
data/
    -train/
          -original/
          -fake/
    -test/
          -original/
          -fake/

In [8]:
original_train_path = path + "data/train/original"
fake_train_path = path + "data/train/fake"
original_test_path = path + "data/test/original"
fake_test_path = path + "data/test/fake"

Function that creates array of filenames

In [9]:
def get_array_of_filenames(original_train_filenames_path, original_test_filenames_path, fake_train_filenames_path, fake_test_filenames_path, main_path):
    ori_train_paths = []
    fake_train_paths = []
    ori_test_paths = []
    fake_test_paths = []

    with open(original_train_filenames_path, 'r') as file:
        ori_train_paths = [main_path + read.strip() for read in file.readlines()]

    with open(fake_train_filenames_path, 'r') as file:
        fake_train_paths = [main_path + read.strip() for read in file.readlines()]

    with open(original_test_filenames_path, 'r') as file:
        ori_test_paths = [main_path + read.strip() for read in file.readlines()]

    with open(fake_test_filenames_path, 'r') as file:
        fake_test_paths = [main_path + read.strip() for read in file.readlines()]

    return ori_train_paths, fake_train_paths, ori_test_paths, fake_test_paths

In [14]:
ori_train, fakes_train, ori_test, fakes_test = get_array_of_filenames(original_train_filenames_path, fake_train_filenames_path, original_test_filenames_path, fake_test_filenames_path, "")

In [15]:
def transformation_to_magnitude(image):
    image = np.mean(image, axis=2)
    dft = np.fft.fft2(image)
    dft_shift = np.fft.fftshift(dft)
    magnitude = np.abs(dft_shift)
    magnitude = magnitude/np.max(magnitude)
    result = np.zeros((magnitude.shape[0], magnitude.shape[1],3))
    result[:,:,0] = magnitude
    result[:,:,1] = magnitude
    result[:,:,2] = magnitude
    return result

In [18]:
def transformation_to_phase(image):

    image = np.mean(image, axis=2)
    dft = np.fft.fft2(image)
    dft_shift = np.fft.fftshift(dft)
    angle = np.angle(dft_shift)
    angle = (angle+np.pi)/(2*np.pi)
    result = np.zeros((angle.shape[0], angle.shape[1],3))
    result[:,:,0] = angle
    result[:,:,1] = angle
    result[:,:,2] = angle
    return result


In [16]:
def transformation_to_everything(image):
    image = np.mean(image, axis=2)
    dft = np.fft.fft2(image)
    dft_shift = np.fft.fftshift(dft)
    magnitude = np.abs(dft_shift)
    magnitude = magnitude/np.max(magnitude)
    angle = np.angle(dft_shift)
    angle = (angle+np.pi)/(2*np.pi)
    result = np.zeros((magnitude.shape[0], magnitude.shape[1],3))
    result[:,:,0] = image
    result[:,:,1] = angle
    result[:,:,2] = magnitude
    return result

Main analisys

In [19]:
K.clear_session()
n_classes = 2
img_width, img_height = 512, 512
train_data_dir = '/content/drive/Shareddrives/datasets/animals/data/train'
validation_data_dir = '/content/drive/Shareddrives/datasets/animals/data/test'
nb_train_samples = 1414 #75750
nb_validation_samples = 142 #25250
batch_size = 16
transformation_function = transformation_to_phase

train_datagen = ImageDataGenerator(
    rescale=1. / 255)
    #preprocessing_function=transformation_function)

test_datagen = ImageDataGenerator(
    rescale=1. / 255)
    #preprocessing_function=transformation_function)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


inception = InceptionV3(weights="imagenet", include_top=False, classes=2)
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(2,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=inception.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='/content/drive/Shareddrives/datasets/results/best_model_animals_batch_16_imagenet.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('/content/drive/Shareddrives/datasets/results/history_animals_batch_16_imagenet.log')

history = model.fit_generator(train_generator,
                    steps_per_epoch = nb_train_samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    epochs=30,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

model.save('/content/drive/Shareddrives/datasets/results/model_trained_animals_batch_16_imagenet.hdf5')

Found 1414 images belonging to 2 classes.
Found 142 images belonging to 2 classes.
87916544/87910968 [==============================] - 0s 0us/step


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
  1/265 [..............................] - ETA: 4:13:17 - loss: 0.8596 - accuracy: 0.4375

KeyboardInterrupt: ignored

How to use model?
First load model:
```
trained_model = models.load_model('path to model')
```
Use this function to load image:

```
def load_file(path):
    img = cv2.imread(path)
    img = img.astype('float64')
    img = img/255.0
    img = img.reshape(-1, 512, 512, 3)
    return img
```


  
Execute predict() method and print results:
```
print(trained_model.predict(load_file(addr2)))
```
Have fun :)
